# Auto Generated Agent Chat: Solving mapping between 2 data schema, execution and debugging

AutoGen offers conversable LLM agents, which can be used to solved various taks with human or automatic feedback, including tasks that require using tools via code. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat). 

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to write code and execute the code. He `AssistantAgent` is a LLM-base agent that can write Python code for a user to excute for a given task. `UsderProxyAgent` is an agent which serves as a proxy for the human user to execute the code written by the `AssistantAgent`. Depending on the setting of `human_input_mode` and `max_consecutive_auto_reply`, the `UserProxyAgent` either solicits feedback from the human user or returns auto-feedback based on the result of code execution (success or failure and corresponding outputs) to `AssistantAgent`. `AssistantAgent` will debug the code and suggest new code if the result contains error. The two agents keep communicating to each other until the task is done.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [ ]:
# %pip install pyautogen>=0.2.3

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file. Ensure to provide GPT-4 model. If the name of the model is not in the list to filter the dict, please add it.


In [2]:
from typing import Dict, Union

from IPython import get_ipython
from IPython.display import Image

import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314", "gpt4-turbo"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the gpt-4 models are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Example: Create the code to generate the mapping between two JSON "schema" (object).

In the example below, let's say we have two JSON objects, `input_schema` and `output_schema`, and we want to generate a mapping between them. This mapping can be useful for data transformation between two REST API for example. We can use AutoGen to generate the code to do the mapping. We are going to see how we can use the agents in AutoGen to write a python script to do the mapping and execute the script.

This process involves:
* Creating a `AssistantAgent` to serve as the assistant to write the code.
* Creating a `UserProxyAgent` to serve as the proxy for the human user to execute the code and to provide feedback to the `AssistantAgent`.

When creating the `UserProxyAgent`, we set `human_input_mode` to `"NEVER"`, which means the `UserProxyAgent` will never solicit feedback from the human user. Instead, it will return auto-feedback based on the result of code execution (success or failure and corresponding outputs) to `AssistantAgent`. We also set `max_consecutive_auto_reply` to `10`, which means the `UserProxyAgent` will stop replying after 10 consecutive auto-feedback or when `is_termination_msg()` returns `True` for the for the received message.

In [3]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=20,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Can you provide a transformation from input schema to output schema?
    ### START: INPUT SCHEMA
    {
        "user": {
            "firstName": string,
            "lastName": string,
            "birthdate": date
        }
    }
    ### END: INPUT SCHEMA

    ### START: OUTPUT SCHEMA
    {
        "fullName": string,
        "age": integer
    }
    ### END: OUTPUT SCHEMA
    """,
)

user_proxy (to assistant):

Can you provide a transformation from input schema to output schema?
    ### START: INPUT SCHEMA
    {
        "user": {
            "firstName": string,
            "lastName": string,
            "birthdate": date
        }
    }
    ### END: INPUT SCHEMA

    ### START: OUTPUT SCHEMA
    {
        "fullName": string,
        "age": integer
    }
    ### END: OUTPUT SCHEMA
    

--------------------------------------------------------------------------------


assistant (to user_proxy):

To transform the input schema to the output schema, we need to perform two main tasks:

1. Concatenate the `firstName` and `lastName` fields to create the `fullName`.
2. Calculate the `age` from the `birthdate`.

Here's a Python function that performs this transformation. The function will take a dictionary matching the input schema and return a dictionary matching the output schema.

```python
# filename: transform_schema.py
from datetime import datetime

def calculate_age(birthdate):
    today = datetime.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

def transform(input_data):
    user = input_data['user']
    full_name = f"{user['firstName']} {user['lastName']}"
    birthdate = datetime.strptime(user['birthdate'], '%Y-%m-%d')
    age = calculate_age(birthdate)
    
    output_data = {
        "fullName": full_name,
        "age": age
    }
    return output_data

# Example usag

In [4]:
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 42,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=20,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    },
)
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""Can you provide a transformation from input API to output API using their OpenAPI definition?
    ### START: INPUT API OPENAPI DEFINITION
    openapi: 3.0.1
    info:
    title: TheCodeBuzz-Service
    version: v1
    paths:
    /weather/{country}/{post_code}/{city}:
        get:
        tags:
            - WeatherForecast
        parameters:
            - in: path
            name: country
            required: true
            schema:
                type: string
            description: The country in which the city is
            - in: path
            name: post_code
            required: true
            schema:
                type: integer
            description: The post-code of the city
            - in: path
            name: city
            required: true
            schema:
                type: string
            description: The city for which the forecast is needed
        responses:
            '200':
            description: Success
            content:
                text/plain:
                schema:
                    type: array
                    items:
                    $ref: '#/components/schemas/WeatherForecast'
                application/json:
                schema:
                    type: array
                    items:
                    $ref: '#/components/schemas/WeatherForecast'
    components:
    schemas:
        WeatherForecast:
        type: object
        properties:
            date:
            type: string
            format: date-time
            description: Date-time in ANSI INCITS 30-1997 format
            temperatureC:
            type: integer
            format: int32
            summary:
            type: string
            nullable: true
            city:
            type: string
            nullable: false
            post_code:
            type: integer
            nullable: false
            country:
            type: string
            nullable: false
            description: The full name of the country
        additionalProperties: false
    securitySchemes:
        bearerAuth:
        type: http
        description: JWT Authorization header using the Bearer scheme
        scheme: bearer
        bearerFormat: JWT
    security:
    - bearerAuth: []
    ### END: INPUT API OPENAPI DEFINITION

    ### START: OUTPUT API OPENAPI DEFINITION
    openapi: 3.0.1
    info:
    title: TheCodeBuzz-Service
    version: v1
    paths:
    /weather:
        get:
        tags:
            - WeatherForecast
        parameters:
            - in: query
            name: longitude
            required: true
            schema:
                type: number
            description: The longitude for which you need the forecast
            - in: query
            name: latitude
            required: true
            schema:
                type: number
            description: The longitude for which you need the forecast
        responses:
            '200':
            description: Success
            content:
                text/plain:
                schema:
                    type: array
                    items:
                    $ref: '#/components/schemas/WeatherForecast'
                application/json:
                schema:
                    type: array
                    items:
                    $ref: '#/components/schemas/WeatherForecast'
    components:
    schemas:
        WeatherForecast:
        type: object
        properties:
            date:
            type: string
            format: date-time
            description: Date time in ISO 8601 format
            temperatureF:
            type: number
            summary:
            type: string
            nullable: true
            longitude:
            type: number
            nullable: false
            latitude:
            type: number
            nullable: false
            countryISO:
            type: string
            nullable: false
        additionalProperties: false
    securitySchemes:
        bearerAuth:
        type: oauth2
        flows:
            implicit:
            authorizationUrl: https://login.mydomain.com/oauth2/v2.0/authorize
            scopes: {}
        description: JWT Authorization header using the Bearer scheme
    security:
    - bearerAuth: []
    ### END: OUTPUT API OPENAPI DEFINITION
    """,
)

user_proxy (to assistant):

Can you provide a transformation from input API to output API using their OpenAPI definition?
    ### START: INPUT API OPENAPI DEFINITION
    openapi: 3.0.1
    info:
    title: TheCodeBuzz-Service
    version: v1
    paths:
    /weather/{country}/{post_code}/{city}:
        get:
        tags:
            - WeatherForecast
        parameters:
            - in: path
            name: country
            required: true
            schema:
                type: string
            description: The country in which the city is
            - in: path
            name: post_code
            required: true
            schema:
                type: integer
            description: The post-code of the city
            - in: path
            name: city
            required: true
            schema:
                type: string
            description: The city for which the forecast is needed
        responses:
            '200':
            description: Success
     

AttributeError: 'NoneType' object has no attribute 'model_dump'